# Data Integration

# TODO:
* some columns require euclidean distance

In [42]:
import pandas as pd
import json
import os
from bs4 import BeautifulSoup
try:
    import pandas_read_xml as pdx
    import tabula
except:
    !pip install pandas_read_xml xlrd tabula-py
    import pandas_read_xml as pdx
import tabula
from functools import reduce
import numpy as np
from math import radians, cos, sin, asin, sqrt, atan2, pi 
import shapefile
from shapely.geometry import Point # Point class
from shapely.geometry import shape # shape() is a function to convert geo objects through the interface
from multiprocessing import Pool
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, Normalizer

In [53]:
# def distance(p1, p2): 
#     lat1, lon1 = p1
#     lat2, lon2 = p2
#     lon1 = radians(lon1) 
#     lon2 = radians(lon2) 
#     lat1 = radians(lat1) 
#     lat2 = radians(lat2) 
       
#     dlon = lon2 - lon1  
#     dlat = lat2 - lat1 
#     a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
#     c = 2 * asin(sqrt(a))  
     
#     r = 6378
#     return(c * r)


r = 6378
deg_len = (2 * pi * r) / 360

# def distance(p1, p2):
#     lat1, lon1 = p1
#     lat2, lon2 = p2
    
#     dlon = lon2 - lon1
#     dlat = lat2 - lat1

#     a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
#     c = 2 * atan2(sqrt(a), sqrt(1 - a))

#     distance = r * c
#     return distance

def distance(p1, p2):
    return np.linalg.norm(p2 - p1) * deg_len

def between_times(time):
    time = [int(i) for i in time.split(':')]
    time_ = 3600 * time[0] + 60 * time[1] + time[2]
    if time_ > 3600 * 7 and time_ < 3600 * 9:
        return True
    return False

NameError: name 'pi' is not defined

In [46]:
print(distance(np.array([-37.9746,145.061]), sc_coordinates[0]))

1323.7307030034021
None


In [3]:
hospitals = open('29893909/hospitals.html')
hospitals = hospitals.read()
hospitals = pd.read_html(hospitals)[0].drop(['Unnamed: 0'], axis=1)

h_id = list(hospitals.id)
lat = [round(i, 6) for i in list(hospitals.lat)]
lng = [round(i, 6) for i in list(hospitals.lng)]
h_coordinates = [np.array((la, ln)) for la, ln in zip(lat, lng)]

In [4]:
supermarkets = pd.read_excel("29893909/supermarkets.xlsx").drop(['Unnamed: 0'], axis=1)
sm_id = list(supermarkets.id)
lat = [round(i, 6) for i in list(supermarkets.lat)]
lng = [round(i, 6) for i in list(supermarkets.lng)]
sm_coordinates = [np.array((la, ln)) for la, ln in zip(lat, lng)]

In [5]:
real_state = open('29893909/real_state.xml').read()[2:-1]
soup = BeautifulSoup(real_state)
tags = [item.name for item in soup.root.childGenerator()]
data = {}
for i, child in enumerate(soup.body.root.children):
    data[tags[i]] = []
    for grandchild in child.children:
        data[tags[i]].append(grandchild.text)
real_state_xml = pd.DataFrame(data, columns=tags)
real_state_json = pd.read_json('29893909/real_state.json')
real_state = real_state_json.append(real_state_xml)

In [6]:
shoppingcenters = tabula.read_pdf('29893909/shopingcenters.pdf', pages='all')
shoppingcenters = reduce(lambda a, b: a.append(b), shoppingcenters)

In [7]:
sc_id = list(shoppingcenters.sc_id)
lat = [round(i, 6) for i in list(shoppingcenters.lat)]
lng = [round(i, 6) for i in list(shoppingcenters.lng)]
sc_coordinates = [np.array((la, ln)) for la, ln in zip(lat, lng)]

In [8]:
stops = pd.read_csv("GTFS - Melbourne Train Information/stops.txt")
ts_id = list(stops.stop_id)
lat = [round(i, 6) for i in list(stops.stop_lat)]
lng = [round(i, 6) for i in list(stops.stop_lon)]
ts_coordinates = [np.array((la, ln)) for la, ln in zip(lat, lng)]

In [9]:
stop_times = pd.read_csv("GTFS - Melbourne Train Information/stop_times.txt")
calendar = pd.read_csv("GTFS - Melbourne Train Information/calendar.txt")
trips = pd.read_csv("GTFS - Melbourne Train Information/trips.txt")
routes = pd.read_csv("GTFS - Melbourne Train Information/routes.txt")

In [10]:
def in_weekdays(service_id):
    temp = 0
    for i in ["monday", "tuesday", "wednesday", "thursday", "friday"]: 
        temp += calendar[calendar.service_id==service_id][i].values[0]
    if temp > 0:
        return True
    return False

In [11]:
def closest(location, id, coords):
    distances = [distance(location, coord) for coord in coords]
    return id[distances.index(min(distances))], min(distances)

def coord(df, index):
    return np.array((
        df.loc[index, 'lat'], 
        df.loc[index, 'lng']
    ))

In [12]:
lrs = len(real_state)
real_state['suburb'] = ["not available"]*lrs
real_state['Shopping_center_id'] = ["not available"]*lrs
real_state['Distance_to_sc'] = [0]*lrs
real_state['Train_station_id'] = ["not available"]*lrs
real_state['Distance_to_train_station'] = [0]*lrs
real_state['travel_min_to_CBD'] = [0]*lrs
real_state['Transfer_flag'] = [-1]*lrs
real_state['Hospital_id'] = ["not available"]*lrs
real_state['Distance_to_hospital'] = [0]*lrs
real_state['Supermarket_id'] = ["not available"]*lrs
real_state['Distance_to_supermarket'] = [0]*lrs

In [13]:
# for index in stop_times.index:
#     if stop_times.arrival_time.str.match("24:\d{1,2}:\d{1,2}"):
#         time = stop_times.loc[index, 'arrival_time'].split(":")
#         time = ":".join(["00", time[1], time[2]])
#         stop_times.loc[index, 'arrival_time'] = time
        
#     if stop_times.departure_time.str.match("24:\d{1,2}:\d{1,2}"):
#          time = stop_times.loc[index, 'departure_time'].split(":")
#          time = ":".join(["00", time[1], time[2]])
#          stop_times.loc[index, 'departure_time'] = time

#     if stop_times.arrival_time.str.match("25:\d{1,2}:\d{1,2}"):
#          time = stop_times.loc[index, 'arrival_time'].split(":")
#          time = ":".join(["00", time[1], time[2]])
#          stop_times.loc[index, 'arrival_time'] = time

#     if stop_times.arrival_time.str.match("25:\d{1,2}:\d{1,2}"):
#          time = stop_times.loc[index, 'departure_time'].split(":")
#          time = ":".join(["00", time[1], time[2]])
#          stop_times.loc[index, 'arrival_time'] = time
     

#     if stop_times.arrival_time.str.match("26:\d{1,2}:\d{1,2}"):
#          time = stop_times.loc[index, 'arrival_time'].split(":")
#          time = ":".join(["00", time[1], time[2]])
#          stop_times.loc[index, 'arrival_time'] = time

#     if stop_times.arrival_time.str.match("26:\d{1,2}:\d{1,2}"):
#          time = stop_times.loc[index, 'departure_time'].split(":")
#          time = ":".join(["00", time[1], time[2]])
#          stop_times.loc[index, 'arrival_time'] = time
             
# stop_times['departure_time'] = pd.to_datetime(stop_times['departure_time'], format='%H:%M:%S').dt.time
# stop_times['arrival_time'] = pd.to_datetime(stop_times['arrival_time'], format='%H:%M:%S').dt.time 

In [14]:
# handling cases where the hour in time > 24
    
for index in stop_times[stop_times.arrival_time.str.match("24:\d{1,2}:\d{1,2}")].index:
    time = stop_times.loc[index, 'arrival_time'].split(":")
    time = ":".join(["00", time[1], time[2]])
    stop_times.loc[index, 'arrival_time'] = time


for index in stop_times[stop_times.departure_time.str.match("24:\d{1,2}:\d{1,2}")].index:
    time = stop_times.loc[index, 'departure_time'].split(":")
    time = ":".join(["00", time[1], time[2]])
    stop_times.loc[index, 'departure_time'] = time
    
for index in stop_times[stop_times.arrival_time.str.match("25:\d{1,2}:\d{1,2}")].index:
    time = stop_times.loc[index, 'arrival_time'].split(":")
    time = ":".join(["01", time[1], time[2]])
    stop_times.loc[index, 'arrival_time'] = time
    
for index in stop_times[stop_times.departure_time.str.match("25:\d{1,2}:\d{1,2}")].index:
    time = stop_times.loc[index, 'departure_time'].split(":")
    time = ":".join(["01", time[1], time[2]])
    stop_times.loc[index, 'departure_time'] = time
    
for index in stop_times[stop_times.arrival_time.str.match("26:\d{1,2}:\d{1,2}")].index:
    time = stop_times.loc[index, 'arrival_time'].split(":")
    time = ":".join(["02", time[1], time[2]])
    stop_times.loc[index, 'arrival_time'] = time
    
for index in stop_times[stop_times.departure_time.str.match("26:\d{1,2}:\d{1,2}")].index:
    time = stop_times.loc[index, 'departure_time'].split(":")
    time = ":".join(["02", time[1], time[2]])
    stop_times.loc[index, 'departure_time'] = time
    
stop_times['departure_time'] = pd.to_datetime(stop_times['departure_time'], format='%H:%M:%S').dt.time
stop_times['arrival_time'] = pd.to_datetime(stop_times['arrival_time'], format='%H:%M:%S').dt.time 

In [15]:
start = pd.to_datetime("07:00:00", format="%H:%M:%S").time()
end = pd.to_datetime("09:00:00", format="%H:%M:%S").time()

stop_times_7_9 = stop_times[(stop_times['departure_time'] > start) & (stop_times['departure_time'] < end) & 
           (stop_times['arrival_time'] > start) &  (stop_times['arrival_time'] < end)]

In [16]:
trips_flinders_street = trips[trips.trip_headsign=='City (Flinders Street)']

In [17]:
# Transfer_flag
transfer_flag = {}
stop_times_trips = pd.merge(left=stop_times_7_9, right=trips_flinders_street, on='trip_id')
for stop in list(stops.stop_id):
    if not stop_times_trips[stop_times_trips.stop_id == stop].empty:
        transfer_flag[stop] = 1
    else:
        transfer_flag[stop] = 0

In [18]:
# Suburb
shp = shapefile.Reader('vic_suburb_boundary/VIC_LOCALITY_POLYGON_shp.dbf') #open the shapefile
all_shapes = shp.shapes() # get all the polygons
all_records = shp.records()     

def get_suburb(inp):
#     point, property_id = inp
    try:
        property_id, point = inp
        for i in range(len(all_shapes)):
            boundary = all_shapes[i] # get a boundary polygon
            if Point(point).within(shape(boundary)):
                return (property_id, all_records[i].as_dict()['VIC_LOCA_2'])
        return ("", "")
    except:
        pass

In [19]:
if 'prop2sub.json' not in os.listdir("."):
    lat = list(real_state.lat)
    lng = list(real_state.lng)
    coords = [(i, j) for i, j in zip(lng, lat)]
    property_ids = list(real_state.property_id)
    inp = [(pid, coord) for pid, coord in zip(property_ids, coords)]
    
    with Pool(4) as p:
        property2suburb = p.map(get_suburb, inp)
else:
    with open('prop2sub.json', 'r') as f:
        property2suburb = json.load(f)
    prop2sub = {}
    for i in property2suburb:
        try:
            prop2sub[i[0]] = i[1]
        except:
            pass

In [20]:
sto = pd.merge(left=stop_times_7_9, right=trips_flinders_street, on='trip_id')

def travel_min_to_CBD(stop_id):
    times = []
    ts = sto[sto.stop_id == stop_id].trip_id.unique()
    for t in ts:
        try:
            st_time = sto[(sto.trip_id == t) & (sto.stop_id == stop_id)].departure_time
            re_time = sto[(sto.trip_id == t) & (sto.stop_id == 19854)].departure_time
            re_time = pd.to_datetime(re_time, format='%H:%M:%S').values[0]
            st_time = pd.to_datetime(st_time, format='%H:%M:%S').values[0]
            times.append(pd.Timedelta(re_time - st_time).seconds / 60)
        except Exception as e:
            pass
    try:
        return (stop_id, round(sum(times) / len(times), 2))
    except:
        pass


In [21]:
%%time
stop_ids = list(stops.stop_id.unique())
with Pool(4) as p:
    stopid2cbd = p.map(travel_min_to_CBD, stop_ids)

CPU times: user 35.1 ms, sys: 59.9 ms, total: 95.1 ms
Wall time: 45 s


In [22]:
stid2cbd = {}
for i in stopid2cbd:
    try:
        stid2cbd[i[0]] = i[1]
    except:
        pass

In [23]:
stid2cbd

{15351: 49.86,
 15353: 45.73,
 19835: 20.46,
 19837: 22.46,
 19838: 24.46,
 19839: 27.23,
 19840: 29.46,
 19841: 292.42,
 19842: 319.22,
 19843: 321.0,
 19844: 66.41,
 19845: 64.41,
 19846: 62.41,
 19847: 37.0,
 19848: 36.0,
 19849: 34.0,
 19850: 32.0,
 19851: 30.0,
 19852: 28.0,
 19853: 23.35,
 19854: 0.0,
 19855: 63.84,
 19856: 61.04,
 19857: 58.22,
 19858: 56.18,
 19859: 53.74,
 19860: 51.74,
 19861: 50.49,
 19862: 48.77,
 19863: 46.87,
 19864: 43.28,
 19865: 41.28,
 19866: 38.77,
 19867: 58.32,
 19868: 55.4,
 19869: 53.4,
 19870: 50.27,
 19871: 46.27,
 19872: 42.44,
 19873: 36.32,
 19874: 34.32,
 19875: 31.56,
 19876: 60.81,
 19877: 55.22,
 19878: 51.22,
 19879: 47.22,
 19880: 81.28,
 19881: 74.22,
 19882: 69.36,
 19883: 67.36,
 19884: 63.26,
 19885: 59.23,
 19886: 67.96,
 19887: 64.96,
 19888: 54.26,
 19889: 51.25,
 19890: 49.25,
 19891: 46.24,
 19892: 24.97,
 19893: 26.9,
 19894: 28.55,
 19895: 29.22,
 19896: 31.9,
 19897: 35.15,
 19898: 34.89,
 19899: 36.92,
 19900: 39.45,
 1990

In [54]:
%%time
for index in real_state.index:
    try:
        lat = real_state.loc[index, 'lat'].values[0]
        lng = real_state.loc[index, 'lng'].values[0]
        location = np.array((lat, lng))
        
        real_state.loc[index, 'suburb'] = prop2sub[real_state.loc[index, 'property_id'].values[0]]
        
        id, dist = closest(location, sc_id, sc_coordinates)
        real_state.loc[index, 'Shopping_center_id'] = id
        real_state.loc[index, 'Distance_to_sc'] = dist
        
        id, dist = closest(location, ts_id, ts_coordinates)
        real_state.loc[index, 'Train_station_id'] = id
        real_state.loc[index, 'Distance_to_train_station'] = dist
        
        real_state.loc[index, 'Transfer_flag'] = transfer_flag[id]
        try:
            real_state.loc[index, 'travel_min_to_CBD'] = stid2cbd[id]
        except:
            real_state.loc[index, 'travel_min_to_CBD'] = 0
            
        
        id, dist = closest(location, h_id, h_coordinates)
        real_state.loc[index, 'Hospital_id'] = id
        real_state.loc[index, 'Distance_to_hospital'] = dist

        id, dist = closest(location, sm_id, sm_coordinates)
        real_state.loc[index, 'Supermarket_id'] = id
        real_state.loc[index, 'Distance_to_supermarket'] = dist
    except:
        pass

CPU times: user 12.4 s, sys: 1.64 ms, total: 12.4 s
Wall time: 12.4 s


In [55]:
real_state

,property_id,lat,lng,addr_street,price,property_type,year,bedrooms,bathrooms,parking_space,...,Shopping_center_id,Distance_to_sc,Train_station_id,Distance_to_train_station,travel_min_to_CBD,Transfer_flag,Hospital_id,Distance_to_hospital,Supermarket_id,Distance_to_supermarket
0,41387,-37.7451,145.065,1/46 Hillside Road,7200000,house,2015,3,2,2,...,SC_017,72.030208,19936,16.512245,28.68,1,hospital_066,67.560544,S_100,92.619168
1,72107,-37.8387,145.263,19 CHURCH STREET,19550000,house,2008,3,2,2,...,SC_032,271.112975,19870,39.096602,50.27,1,hospital_002,112.191911,S_139,37.686053
2,51703,-37.7807,145.124,84 Williamsons Road,9630000,house,2013,3,1,2,...,SC_091,42.295377,20042,246.049947,30.26,1,hospital_194,73.494809,S_188,39.967390
3,37969,-37.7756,145.02,126 Arthur Street,7380000,house,2014,3,2,0,...,SC_001,150.084047,19930,27.951684,18.80,1,hospital_029,92.459173,S_219,148.972603
4,92396,-37.9746,145.061,68 Latrobe street,12558000,house,2015,3,1,2,...,SC_003,172.472211,19865,52.272179,41.28,1,hospital_133,54.405879,S_011,45.005164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,24209,-37.697565000000004,144.97390900000002,24 Beccles Street,7684000,house,2011,3,1,1,...,SC_041,144.082023,19941,34.034415,34.12,1,hospital_126,161.131536,S_107,107.158047
997,80370,-37.986048,145.158073,56 Putt Grove,9975000,house,2016,4,2,2,...,SC_032,251.786170,19871,162.703026,46.27,1,hospital_179,46.673778,S_226,130.774170
998,75144,-37.89724731,145.0753326,4/26A Howe Street,3618000,house,2009,2,1,1,...,SC_017,266.769010,19984,57.269231,34.71,1,hospital_075,142.811755,S_020,117.864896
999,80699,-37.97064972,145.2060394,10 McNab Court,3036000,house,2008,3,1,1,...,SC_062,234.697039,45795,117.205576,46.43,1,hospital_053,205.176238,S_096,162.113167


In [51]:
real_state

,property_id,lat,lng,addr_street,price,property_type,year,bedrooms,bathrooms,parking_space,...,Shopping_center_id,Distance_to_sc,Train_station_id,Distance_to_train_station,travel_min_to_CBD,Transfer_flag,Hospital_id,Distance_to_hospital,Supermarket_id,Distance_to_supermarket
0,41387,-37.7451,145.065,1/46 Hillside Road,7200000,house,2015,3,2,2,...,SC_017,72.030208,19936,16.512245,28.68,1,hospital_066,67.560544,S_100,92.619168
1,72107,-37.8387,145.263,19 CHURCH STREET,19550000,house,2008,3,2,2,...,SC_032,271.112975,19870,39.096602,50.27,1,hospital_002,112.191911,S_139,37.686053
2,51703,-37.7807,145.124,84 Williamsons Road,9630000,house,2013,3,1,2,...,SC_091,42.295377,20042,246.049947,30.26,1,hospital_194,73.494809,S_188,39.967390
3,37969,-37.7756,145.02,126 Arthur Street,7380000,house,2014,3,2,0,...,SC_001,150.084047,19930,27.951684,18.80,1,hospital_029,92.459173,S_219,148.972603
4,92396,-37.9746,145.061,68 Latrobe street,12558000,house,2015,3,1,2,...,SC_003,172.472211,19865,52.272179,41.28,1,hospital_133,54.405879,S_011,45.005164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,24209,-37.697565000000004,144.97390900000002,24 Beccles Street,7684000,house,2011,3,1,1,...,SC_041,144.082023,19941,34.034415,34.12,1,hospital_126,161.131536,S_107,107.158047
997,80370,-37.986048,145.158073,56 Putt Grove,9975000,house,2016,4,2,2,...,SC_032,251.786170,19871,162.703026,46.27,1,hospital_179,46.673778,S_226,130.774170
998,75144,-37.89724731,145.0753326,4/26A Howe Street,3618000,house,2009,2,1,1,...,SC_017,266.769010,19984,57.269231,34.71,1,hospital_075,142.811755,S_020,117.864896
999,80699,-37.97064972,145.2060394,10 McNab Court,3036000,house,2008,3,1,1,...,SC_062,234.697039,45795,117.205576,46.43,1,hospital_053,205.176238,S_096,162.113167


# Data Reshaping

## Standardization

In [26]:
# standard_scaler = StandardScaler()
# real_state_standard = real_state.copy()
# columns = ['price', 'Distance_to_sc', 'Distance_to_hospital']
# real_state_standard[columns] = standard_scaler.fit_transform(real_state_standard[columns])

In [27]:
# real_state_standard

In [28]:
# minmax_scaler = MinMaxScaler()
# real_state_minmax = real_state.copy()
# real_state_minmax[columns] = standard_scaler.fit_transform(real_state_minmax[columns])

In [29]:
# real_state_minmax

In [30]:
# log_normalized = real_state.copy()
# for index in log_normalized.index:
    
#     for column in columns:
#         val = log_normalized.loc[index, column]
#         try:
#             log_normalized.loc[index, column] = np.log(val)
#         except:
#             log_normalized.loc[index, column] = np.log(val.values[0])

In [31]:
# real_state

In [32]:
# real_state_boxcox = real_state.copy()
# pt = PowerTransformer(method='box-cox')
# normalizer = Normalizer()

# real_state_boxcox[[columns]] = normalizer.fit_transform(real_state_boxcox[columns])
# # real_state_boxcox[[columns]] = pt.fit_transform(real_state_boxcox[columns])
        